In [1]:
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import climakitae as ck

print('done')

done


In [2]:
variables_units = ("Fosberg fire weather index","[0 to 100]","Dynamical","hourly",[2.0])
variable, unit,downscale,timescale,GWL = variables_units
print(variable,unit,downscale,timescale,GWL)
print(f"Processing variable: {variable}")
ds = get_data(
    variable=variable,
    # units=unit,
    downscaling_method=downscale,
    resolution="3 km",
    timescale=timescale,
    cached_area="Southern California Edison",
    approach="Warming Level",
    warming_level_window=15,
    warming_level=GWL
)


Fosberg fire weather index [0 to 100] Dynamical hourly [2.0]
Processing variable: Fosberg fire weather index
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [3]:
hours_in_year = 365 * 24
# Calculate the hour of the year
hour_of_year = ds['time_delta'].values % hours_in_year
# Assign the new hour_of_year coordinate to the dataset
ds = ds.assign_coords(hour_of_year=('time_delta', hour_of_year))
print(ds)
ds = ds.drop_vars(['landmask', 'lakemask'])
hourly_mean = ds.groupby('hour_of_year').mean(dim='time_delta', skipna=True)
print(hourly_mean)
del ds
hours_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) * 24
cumulative_hours = np.cumsum(hours_in_month)
# Function to map hour_of_year to month
def hour_to_month(hour):
    return np.searchsorted(cumulative_hours, hour) + 1

# Apply the function to calculate the month for each hour_of_year

print(hourly_mean)
months = hour_to_month(hourly_mean['hour_of_year'].values)
hourly_mean = hourly_mean.assign_coords(month=('hour_of_year', months))

monthly_max = hourly_mean.groupby('month').max(dim='hour_of_year', skipna=True)
print(monthly_max)

<xarray.DataArray np.str_('Fosberg fire weather index') (warming_level: 1,
                                                         time_delta: 262800,
                                                         y: 240, x: 149,
                                                         simulation: 8)> Size: 301GB
dask.array<getitem, shape=(1, 262800, 240, 149, 8), dtype=float32, chunksize=(1, 1608, 54, 33, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level      (warming_level) float64 8B 2.0
  * x                  (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y                  (y) float64 2kB 5.899e+05 5.929e+05 ... 1.307e+06
    lakemask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 143kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 143kB 31.49 31.5 31.52 ... 39.19 39.21
    lon                (y, x) float32 143kB -117.7 -117.7 ... -118.2 -118.2
    Lambert_Conformal  int64 8B 0
  *

In [4]:
percentiles = [10,50,90]
percentile_values_ts = {p: monthly_max.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}
print(percentile_values_ts)

{10: <xarray.DataArray np.str_('Fosberg fire weather index') (warming_level: 1,
                                                         month: 12, y: 240,
                                                         x: 149)> Size: 3MB
dask.array<getitem, shape=(1, 12, 240, 149), dtype=float64, chunksize=(1, 9, 240, 149), chunktype=numpy.ndarray>
Coordinates:
  * warming_level  (warming_level) float64 8B 2.0
  * x              (x) float64 1kB -4.236e+06 -4.233e+06 ... -3.792e+06
  * y              (y) float64 2kB 5.899e+05 5.929e+05 ... 1.304e+06 1.307e+06
    lat            (y, x) float32 143kB 31.49 31.5 31.52 ... 39.18 39.19 39.21
    lon            (y, x) float32 143kB -117.7 -117.7 -117.7 ... -118.2 -118.2
  * month          (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
    quantile       float64 8B 0.1
Attributes:
    variable_id:           fosberg_index_derived
    extended_description:  Fire weather index using meteorological inputs (te...
    units:                 [0 to 100]
    d

In [ ]:
ck.export(percentile_values_ts[50], "Fosberg fire weather index_GWL2.0_8760_climo_GWL2.0_50th.nc", "NetCDF")

print('done')

Exporting specified data to NetCDF...
Saving file locally as NetCDF4...


In [ ]:
print('done')